# Analyzing UMD Professor Salary and Review
### By Sean Huang and Tochukwu Ibe-Ekeocha


## Introduction

In this project we want to analyze a two main datasets from the University of Maryland College Park. The first is data about the salaries of all the professors at the University as well as Student created reviews of classes taught by the professors. Our goal it to use a number of tools and methodologies from the data science pipeline to use the datasets to make meaningful insights about the relationship between professor salary and student course review.

In [ ]:
import requests as req
import pandas as pd
import sqlite3 as sql
import requests
from functools import reduce
import numpy as np
import os.path
from os import path
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Data  Collection

To start, we first need some data on both the salaries of each professors in the university as well as their popularity. Since popularity is somewhat abstract and subjective, we decided to use the review that they recieved from their past students. For the salary data, we found a website called the Diamondback Salary Guide(https://salaryguide.dbknews.com/), a platform that allows the user to look up any university staff and get their salary, department, and position. 

In [ ]:
db = sql.connect(":memory:")
sqlite_file = 'salaries_data.sql'
sql_file = open(sqlite_file)
sql_as_string = sql_file. read()
db. executescript(sql_as_string)

In [ ]:
salary_query= """
SELECT 
 firstname, lastname, department, salary, position
FROM Salaries
"""

salaries = pd.read_sql(salary_query, db)

salaries.head()

For the review data, we used PlanetTerp, the official review site for UMD professors. Since the process of pulling every single professor from PlanetTerp takes a long time, we decided to check if there is a local database we can use before calling the API, and just using the local data if one is found. If one is not found, we fetch the professors 1000 at a time, using the API. After we fetch them, we save it locally so we don't have to do wait next time.

In [ ]:
def fetch_profs(): 
    profs = dict()
    matched_profs = dict()

    for i in range(0, 10000, 1000):
        r = requests.get('https://api.planetterp.com/v1/professors?reviews=true', params={
          'limit': '1000',
          'offset': i,
        }, headers = {'Accept': 'application/json'})

        for e in r.json():
            profs[e.get("slug")] = e
            matched_profs[e.get("slug")] = None
    return (profs, matched_profs)

In [ ]:
def fetch_reviewsData(_data, ids):
    if(path.isfile("reviews.csv")):
        return pd.read_csv("reviews.csv")
        
    reviews_data = []
    i = 0
    for v in _data:

        r = requests.get('https://api.planetterp.com/v1/professor', params={
          'name': v,  'reviews': 'true'
        }, headers = {'Accept': 'application/json'}).json()


        slug = r.get("slug")
        revs = r.get("reviews")
        if(revs == None):
            continue

        for rev in revs:  

            reviews_data.append([ 
                slug,
                rev.get("rating"),  
                rev.get("expected_grade"), 
                rev.get("course"),
                rev.get("review"),
                ids[i]
            ])
            
        i += 1;
    reviews_df = pd.DataFrame(reviews_data, columns=['slug', 'rating', 'expected_grade', 'course', 'review', "prof_df_id"])
    reviews_df.to_csv(path_or_buf="reviews.csv", index=False)
    return  reviews_df

## Data management

In [ ]:
def fetch_matched_salaries():
    if(path.isfile("matched_salaries.csv")):
        return pd.read_csv("matched_salaries.csv")
    
    
    (profs, matched_profs) = fetch_profs()
    
    arr = list(set(salaries["department"]))
    for dep in arr:   
        get_department(dep, profs, matched_profs)
        
    data = [ list(v)  for v in matched_profs.values() if v != None]
    matched_salaries = pd.DataFrame(data, columns=['firstname', 'lastname', 'department', 'salary', "position", "avg_rating", 'slug', "planetTerpName", "courses", "num_courses"])
    matched_salaries.to_csv(path_or_buf="matched_salaries.csv", index=False)
    return matched_salaries


We now have two data sets with information on UMD's profesors. Since they came from two different places and uses different format, we will have to merge them manually. To do this, we wrote a function called find_closest_match(), which find the closest match for a name in a dataframe.

In [ ]:
def find_closest_match(name, matches):
    firstname = name.split(" ")[0]
    ret = [ m  for m in matches if m[0].find(firstname) >= 0 ]
    if(len(ret) > 0):
        return ret[0]
    return None
def calc_avg_review(revs):
    length = len(revs)
    if(length == 0):
        return np.nan
    total = 0
    for r in revs:
        total += r.get("rating")
    return total / length
        

With a simple helper method, we are also able to parse each professor's information into columns in a dataframe.

In [ ]:
def get_department(dep, profs, matched_profs):
    matches = dict()

    for prof in profs.values():
        
        slug = prof.get("slug")
        name = prof.get("name")
        courses = prof.get("courses")
        avg_rating = calc_avg_review(prof.get("reviews"))
        
        if(matched_profs.get(slug) != None):
            continue

        lastname = name.split(" ")[-1]

        salary_query= f"""
            SELECT 
            firstname, 
            lastname,
            department,
            salary,
            position
            FROM Salaries
            WHERE lastname like "{lastname}"
            AND department = "{dep}"
        """
        rows = list(db.execute(salary_query))
        matches[lastname] = (len(rows), rows, name, slug, courses, avg_rating)
        

    for match in matches.keys():
        (length, v, name, slug, courses, avg_rating) = matches.get(match)
        if(length > 0):
           
            value = find_closest_match(name, v)
         
            if(value != None):
                
                matched_profs[slug] = (
                    value[0],
                    value[1],
                    value[2],
                    value[3],
                    value[4],
                    avg_rating,
                    slug,
                    name,
                    "-".join(courses),
                    len(courses)
                )
                

    return

Since we are only looking into professors, we filter the database to only inclue those with either "Lecturer" or "Professor" in their position. 

In [ ]:
matched_salaries = fetch_matched_salaries()
matched_salaries = matched_salaries[matched_salaries['position'].str.contains("Professor") | matched_salaries['position'].str.contains("Lecturer")]
matched_salaries.head()

In [ ]:
reviews_df = fetch_reviewsData(list(matched_salaries["planetTerpName"]), matched_salaries.index.values)
reviews_df.head()

## Exploratory data analysis

In this section, we will start to take a look at the dataset and try to find patterns and correlations that could be relevant and used to form our hypotheses later. To start, we take a look at how a professor's review correlates with the professor's salary.

In [ ]:
figsize = (20, 11)
prof_salary = matched_salaries.dropna(subset = ['avg_rating'])
prof_salary = prof_salary[prof_salary['position'].str.contains("Professor") | prof_salary['position'].str.contains("Lecturer")]

plt.figure(figsize=figsize)
plt.scatter(prof_salary['salary'], prof_salary['avg_rating'])
plt.xlabel('Salary')
plt.ylabel('Average Rating')

m,b = np.polyfit(prof_salary['salary'], prof_salary['avg_rating'], 1)
plt.plot(prof_salary['salary'], m * prof_salary['salary'] + b, color = 'red')
plt.title("Salary vs Average Rate by Professsors")
plt.show()

From this graph, we are able to see that there is a negative correlation with a professor's salary and the average review the professor has. This indicates that professors that are paid more are more likely to have a lower average review on PlanetTerp. 

We also wanted to see if there is a big difference between the average salary of professors in each department. So, we grouped the professor's salary data by department and found the average salary of each department's professors. We also sorted them to see the top salaries. 

In [ ]:
dept_avg = matched_salaries.groupby("department").mean().sort_values(by=["salary"], ascending=False)
dept_avg = dept_avg.dropna()
dept_avg

According to the dataframe, we can see the top department for average salary being the Business School, the school of Behavioral and Social Science, and the Academy of Innovation & Entrepreneurship. The three schools that gets paid the least are the school of Archeology, Office of Undergraduate Studies, and the school of Enviornmental Science.

In [ ]:
plt.figure(figsize=figsize)
for i in dept_avg.index:
    plt.scatter(dept_avg.loc[i]['salary'], dept_avg.loc[i]['avg_rating'])
    plt.annotate(i, (dept_avg.loc[i]['salary'], dept_avg.loc[i]['avg_rating']))
    
plt.title('Salary vs Average Rating by Departments')
plt.xlabel('Salary')
plt.ylabel('Average Rating')

m,b = np.polyfit(dept_avg['salary'], dept_avg['avg_rating'], 1)


plt.plot(dept_avg['salary'], m * dept_avg['salary'] + b)

plt.show()

Just like above, we see the same negative correlation between salary and average rating. However, this graph also brings us information about departments that are outliers. For example, the school of Archeology gets a particularly low rating for its average salary and the the school of Academy of Innovation & Entrepreneurship gets high rating for its salary.

We also wanted to see if the amount of unique courses a professor has taught also correlates with the amount of salary that professor earned. So, we created another scatterplot to check it.

In [ ]:
prof_salary = matched_salaries.dropna(subset = ['avg_rating'])
plt.figure(figsize=figsize)

plt.scatter(prof_salary['num_courses'], prof_salary['salary'])
plt.xlabel('Number of Courses Taught')
plt.ylabel('Salary')
plt.title('Number of Courses Taught vs Salary by Professors')

m,b = np.polyfit(prof_salary['num_courses'], prof_salary['salary'], 1)
plt.plot(prof_salary['num_courses'], m * prof_salary['num_courses'] + b, color = 'red')
plt.show()

The graph does show a positive correlation between the two, indicating that a professor that has taught many different courses are more likely to have a higher salary.

### Finding general correlation

To get a better look of what correlation exists in the dataset, we use a correlation matrix to match each columns with all the other columns

In [ ]:
plt.subplots(figsize = figsize)
g = sns.heatmap(matched_salaries.corr(), annot = True, cmap = "YlGnBu", fmt = '.1g', linewidths=.5,)

g.set_title("Correlation Matrix between Columns")
g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')
g.set_yticklabels(g.get_yticklabels(), rotation=45, horizontalalignment='right')

There doesn't seem to be any significant correlation with any of the numerical data so far. We decided to include the department data to find any correlations for it.

Since the departments are categorical data, we decide to use the get_dummies function from pandas to convert it to numerical data. The new dataframe include a new column for each of the categories in the department column, and indicates which category it belongs to with a 1 in the department a row belongs to and 0s in all other columns.

In [ ]:
dept_features = pd.get_dummies(matched_salaries['department'], drop_first = True, prefix = 'dept')
std_features = matched_salaries[['avg_rating', 'salary', 'num_courses']]

#combine our data
comb_features = pd.concat([std_features, dept_features], axis = 1)

With the new dummies, we take a look at our updated matrix.

In [ ]:
plt.subplots(figsize = figsize)
g = sns.heatmap(comb_features.corr(), annot = True, cmap = "YlGnBu", fmt = '.1g', linewidths=.5,)

g.set_title("Correlation Matrix between Columns")
g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')
g.set_yticklabels(g.get_yticklabels(), rotation=45, horizontalalignment='right')
g.plot()

Unfortnunaly, there are still no significant correlation between our data. The two strongest correlations are there are dept_BMGT with salary and dept_ARHU with num_courses. This indicatest that a professor in BMGT is more likely to get a higher salary and a professor in ARHU is more likely to teach many different classes.

## Machine Learning

### MODEL 1

We first try to use a linear regression model to predict a professors salary. Knowing the number of unique courses taught, average rating, and department for each of the professors, we can predict what their salary should be.

In order to add more context to the data, we need a way to represent relationships between columns in our dataframe. We use the new dataframe we got above to create two interaction terms by multiplying the department column by the number of courses as well as the rating.

In [ ]:
comb_features = comb_features.dropna()
for i in filter(lambda i : i!= 'AGNR', matched_salaries['department'].unique()):
    comb_features['num_course*' + i] = comb_features['dept_' + i]*comb_features['num_courses']
    comb_features['rating*' + i] = comb_features['dept_' + i]*comb_features['avg_rating']
    
comb_features

We fit our data on a linear regression model, where X is the independent variables, which includes all the columns in the dataframe except salary, and Y the dependent variable, which is the salary.

In [ ]:
#seperate our data
X = comb_features.drop('salary', axis = 1)
y = comb_features['salary']
reg = LinearRegression()
reg.fit(X,y)


After trainng the model, we used it to make predictions on every professors that have the independent variables in our dataframe.

In [ ]:
predictions = []
for i, row in matched_salaries.iterrows():
    if i in comb_features.index:
        predictions.append(round(reg.predict([X.loc[i]])[0],2))
        
    else :
        predictions.append(np.nan)
matched_salaries['predicted_salary'] = predictions
matched_salaries.head()

To see how accurate our model is, we make a scatter plot of the predicted salary of each professor vs the actual salary of each professor.

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(matched_salaries['predicted_salary'], matched_salaries['salary'], color="red")

plt.xlabel('Predicted Salary')
plt.ylabel('Actual Salary')
plt.title('Predicted Salary vs Actual Salary')

x = prof_salary['avg_rating']
y = prof_salary['salary']

plt.plot([0,500000],[0,500000])

plt.show()

From the graph, we see that our predictions do follow the same trend as the actual salaries of the professor, especually with professors with lower income. However, there are a few notable cases of under-prediction outliers when it comes to professors with a very high income.

### MODEL 2

Next we want to use salary, average rating, and department for each student's review to get a lot to predict the grades they got for that class. 

In [ ]:
def get_salary(x): 
    return matched_salaries.loc[x.prof_df_id]["salary"]
def get_department(x):
    return matched_salaries.loc[x.prof_df_id]["department"]
    
def map_grades(g): 
    if(pd.isnull(g)):
        return np.nan
    
    d = {'A': 90, "B": 80, "C": 70, "D": 60, "F": 50}
    match = re.search('[a-z|A-Z]', g, re.IGNORECASE)
    if(match == None):
        return np.nan
    val = d.get(match.group(0))
    return val if val != None else np.nan

# reviews_df['course_num'] = reviews_df.apply(lambda x: convert_to_num_rep(x.course), axis=1)
reviews_df['grade_num'] = reviews_df.apply(lambda x: map_grades(x.expected_grade), axis=1)
reviews_df['department'] = reviews_df.apply(lambda x: get_department(x), axis=1)
reviews_df['salary'] = reviews_df.apply(lambda x: get_salary(x), axis=1)
reviews_df.head()

Since grades on a review are given in a letter format, we first convert it into numeric data using an approximation of the precentage the student got as a final grade. We also drop all the reviews without a grade.

In [ ]:
non_missing_grades = reviews_df.dropna()
non_missing_grades

For this model, we decided to seperate the data into trainng data and testing data. This means we use the training data to create the model and test the model against the testing data. This strategy helps us to avoid overfitting.

In [ ]:
[train_sample,  test_sample] = train_test_split(non_missing_grades)

Using the same process above, we create interactions terms for the categorical data of department and 

In [ ]:
def get_dummies(df):
    dept_features = pd.get_dummies(df['department'], drop_first = True, prefix = 'dept')
    std_features = df[['rating', 'grade_num', 'salary']]

    comb_features = pd.concat([std_features, dept_features], axis = 1)

    for i in filter(lambda i : i!= 'AGNR', non_missing_grades['department'].unique()):
        if i in comb_features.index:
            comb_features['salary*' + i] = comb_features['dept_' + i]*comb_features['salary']
            comb_features['rating*' + i] = comb_features['dept_' + i]*comb_features['rating']
        else:
            comb_features['salary*' + i] = 0
            comb_features['rating*' + i] = 0
    return comb_features

In [ ]:
training_features = get_dummies(train_sample)

In [ ]:
X2 = training_features.drop('grade_num', axis = 1)
y2 = training_features['grade_num']
reg2 = LinearRegression()
reg2.fit(X2,y2)

In [ ]:
predictions = []
testing_features = get_dummies(test_sample)
X2 = testing_features.drop('grade_num', axis = 1)
y2 = training_features['grade_num']

for i, row in X2.iterrows():
    predictions.append(round(reg2.predict([X2.loc[i]])[0],2))
        
testing_features['predicted_grade'] = predictions
testing_features.head()

We decide to make a plot of ratings vs grades for both the actual grades and the predicted grade to see they follow the same trend.

In [ ]:
plt.figure(figsize=figsize)
plt.scatter(testing_fetures['rating'], testing_fetures['grade_num'], color="red", label = 'Actual Grade')
plt.scatter(testing_fetures['rating'], testing_fetures['predicted_grade'], color="blue", label = 'Predicted Grade')
plt.xlabel('Rating')
plt.ylabel('Grade')

plt.title('Rating vs Grades for Predicted Grades and Actual Grades')

plt.legend()

x = testing_fetures['rating']
y = testing_fetures['predicted_grade']

plt.show()

From the graph, we are able to see while the actual grades varies a lot more than our predicted grades, they do all follow the same trend of correlating positively with rating.

With the trained model, we can predict the grades of reviews with missing expected grades.

In [ ]:
predictions = []
missing_features = get_dummies(reviews_df)
X2 = missing_features.drop('grade_num', axis = 1)
y2 = missing_features['grade_num']

for i, row in X2.iterrows():
    predictions.append(round(reg2.predict([X2.loc[i]])[0],2))
        
reviews_df['predicted_grade'] = predictions

In [ ]:
reviews_df[pd.isnull(reviews_df["grade_num"])]

## Conclusion

With this turorial, we analyzed the salary and review data for each of the professors in UMD. From the data analysis and visualizations we created, we discovered that the salary correlates negatively with a professor's rating and positively with the amount of different courses a professor has taught. We are also able to see that the department a professor is in plays a role in the expected salary of a professor and the expected amount of different classes a professor has to teach. With these new insights, we created two models that can predict the salary of a professor based on the professor's rating, department, and number of courses the professor teaches, as well as predict the grade of a student in a class based on the rating the student left, the salary of the professor and the department of the course.